In [ ]:
pip install -U flash-attn --no-build-isolation

In [1]:
import re

import torch
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [2]:
model_name = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLM(model=model_name, max_num_batched_tokens=8192, seed=1, trust_remote_code=True)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-30 16:06:54 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=1)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-30 16:06:55 utils.py:608] Found nccl from library /home/ubuntu/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 09-30 16:06:57 selector.py:28] Using FlashAttention backend.
INFO 09-30 16:06:58 weight_utils.py:193] Using model weights format ['*.safetensors']


model-00002-of-000002.safetensors:  22%|##1       | 1.13G/5.23G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:  70%|#######   | 6.04G/8.59G [00:00<?, ?B/s]

INFO 09-30 16:11:31 model_runner.py:173] Loading model weights took 12.8725 GB
INFO 09-30 16:11:32 gpu_executor.py:119] # GPU blocks: 2858, # CPU blocks: 546
INFO 09-30 16:11:35 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-30 16:11:35 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-30 16:11:42 model_runner.py:1057] Graph capturing finished in 7 secs.


In [ ]:
prompt = r'''Complete the following Lean 4 code:

```lean4
'''

code_prefix = r'''import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
'''

In [ ]:
sampling_params = SamplingParams(
    temperature=1.0,
    max_tokens=200,
    top_p=0.95,
    n=10,
)

In [ ]:
model_inputs = [prompt + code_prefix]

In [ ]:
model_outputs = model.generate(
    model_inputs,
    sampling_params,
    use_tqdm=False,
)